In [1]:
pip install streamlit streamlit-folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.8 MB/s eta 0:00:00


In [2]:
import streamlit as st
import folium
from streamlit_folium import st_folium
import joblib
import numpy as np
import ee
import datetime

# --- Initialize Earth Engine ---
ee.Authenticate()
ee.Initialize(project='ringed-trail-454308-d2')

2025-06-21 07:41:35.505 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [3]:
import geopandas as gpd
import json
from shapely.geometry import shape

In [4]:
code='''
import joblib
import streamlit as st
import folium
from streamlit_folium import st_folium
import numpy as np
import geopandas as gpd
import datetime
import ee
from shapely.geometry import shape

# Initialize Earth Engine
ee.Initialize(project='ringed-trail-454308-d2')

# Load trained ensemble model and scaler
model = joblib.load("this_ensemble_model.pkl")
scaler = joblib.load("this_scaler.pkl")

# UI setup
st.set_page_config(layout="wide")
st.title("🧠 Deforestation Prediction Grid Map (Maharashtra Region)")

@st.cache_data
def load_grid():
    return gpd.read_file("Maharashtra_5x5km_Grid.geojson")

gdf = load_grid()
gdf["predicted"] = None

# Toggle prediction color overlay
enable_colors = st.checkbox("🟩🟥 Show prediction color (green: forest, red: deforested)", value=True)

# Center map around Maharashtra
m = folium.Map(location=[19.5, 74], zoom_start=7.5, control_scale=True)

# Add grid cells with dynamic styling
for _, row in gdf.iterrows():
    pred = row["predicted"]
    fill = "gray"
    if enable_colors and pred in [0, 1]:
        fill = "green" if pred == 0 else "red"

    folium.GeoJson(
        row["geometry"],
        style_function=lambda x, fill_color=fill: {
            "fillColor": fill_color,
            "color": "black",
            "weight": 0.5,
            "fillOpacity": 0.6 if enable_colors else 0.1
        }
    ).add_to(m)

st.markdown("### 📍 Click on a grid cell to get deforestation prediction")
map_data = st_folium(m, height=600, width=900)

if map_data.get("last_clicked"):
    click_point = shape({
        "type": "Point",
        "coordinates": [map_data["last_clicked"]["lng"], map_data["last_clicked"]["lat"]]
    })

    selected = gdf[gdf.geometry.contains(click_point)]
    if selected.empty:
        st.warning("⚠️ Click within a valid grid cell.")
    else:
        cell_idx = selected.index[0]
        cell = selected.iloc[0]
        lon, lat = cell.geometry.centroid.x, cell.geometry.centroid.y
        st.info(f"🧭 Selected Cell Center: ({lat:.4f}, {lon:.4f})")

        point = ee.Geometry.Point([lon, lat])
        try:
            end_date = datetime.datetime.now().date()
            start_date = end_date - datetime.timedelta(days=30)

            # NDVI & EVI
            mod13 = ee.ImageCollection("MODIS/061/MOD13Q1") \
                .filterBounds(point).filterDate(str(start_date), str(end_date)) \
                .select(["NDVI", "EVI"]).mean()
            ndvi = mod13.select("NDVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("NDVI").getInfo()
            evi = mod13.select("EVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("EVI").getInfo()

            # NDMI
            def compute_ndmi(img):
                return img.addBands(img.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']).rename("NDMI"))

            ndmi_img = ee.ImageCollection("MODIS/061/MOD09GA") \
                .filterBounds(point).filterDate(str(start_date), str(end_date)) \
                .map(compute_ndmi).select("NDMI").mean()
            ndmi = ndmi_img.reduceRegion(ee.Reducer.mean(), point, 500).get("NDMI").getInfo()

            # Precipitation
            precip = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
                .filterDate(str(start_date), str(end_date)) \
                .select("precipitation").sum().reduceRegion(ee.Reducer.mean(), point, 5000).get("precipitation").getInfo()

            # LST
            lst_img = ee.ImageCollection("MODIS/061/MOD11A1") \
                .filterDate(str(start_date), str(end_date)) \
                .select("LST_Day_1km").mean()
            lst = lst_img.multiply(0.02).subtract(273.15).reduceRegion(ee.Reducer.mean(), point, 1000).get("LST_Day_1km").getInfo()

            # Treecover (static or annual)
            treecover_img = ee.ImageCollection("MODIS/061/MOD44B") \
                .filterBounds(point).select("Percent_Tree_Cover").mean()
            treecover = treecover_img.reduceRegion(ee.Reducer.mean(), point, 250).get("Percent_Tree_Cover").getInfo()

            # Seasonal anomaly
            month = end_date.month
            # These would ideally come from precomputed monthly means
            lst_season_mean = 30  # placeholder
            precip_season_mean = 50  # placeholder
            lst_anomaly = lst - lst_season_mean
            precip_anomaly = precip - precip_season_mean

            features = np.array([[evi, ndmi, lst_anomaly, precip_anomaly, treecover]])
            scaled = scaler.transform(features)
            pred = model.predict(scaled)[0]
            label = "Deforested" if pred == 1 else "Forest"

            gdf.at[cell_idx, "predicted"] = pred

            st.success(f"✅ Prediction: {label}")
            st.markdown("📊 **Feature Summary**")
            st.table({
                "NDVI": [round(ndvi, 4)],
                "EVI": [round(evi, 4)],
                "NDMI": [round(ndmi, 4)],
                "LST (°C)": [round(lst, 2)],
                "LST Anomaly": [round(lst_anomaly, 2)],
                "Precipitation (mm)": [round(precip, 2)],
                "Precip Anomaly": [round(precip_anomaly, 2)],
                "Treecover": [round(treecover, 1)]
            })

        except Exception as e:
            st.error("❌ Error during prediction.")
            st.text(str(e))

'''

In [5]:
with open("app.py", "w") as f:
    f.write(code)

In [6]:
!wget -q -O - ipv4.icanhazip.com


35.237.115.225


In [7]:
!streamlit run app.py & npx localtunnel --port 8501



⠙
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.115.225:8501

⠹⠸⠼⠴⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹your url is: https://upset-snakes-hope.loca.lt
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not 

In [12]:
code2 = '''
import joblib
import streamlit as st
import folium
from streamlit_folium import st_folium
import numpy as np
import geopandas as gpd
import datetime
import ee

# Initialize Earth Engine
ee.Initialize(project='ringed-trail-454308-d2')

# Load trained ensemble model and scaler
model = joblib.load("this_ensemble_model.pkl")
scaler = joblib.load("this_scaler.pkl")

# UI setup
st.set_page_config(layout="wide")
st.title("🧠 Deforestation Prediction Grid Map (Maharashtra Region)")

@st.cache_data
def load_grid():
    return gpd.read_file("Maharashtra_5x5km_Grid.geojson")

# 🔁 Limit to first 10 cells for faster testing
gdf = load_grid().head(10)

# Prediction date range
end_date = datetime.datetime.now().date()
start_date = end_date - datetime.timedelta(days=30)

# Placeholder seasonal averages
lst_season_mean = 30
precip_season_mean = 50

def compute_features(lon, lat):
    point = ee.Geometry.Point([lon, lat])
    try:
        # NDVI & EVI
        mod13 = ee.ImageCollection("MODIS/061/MOD13Q1") \\
            .filterBounds(point).filterDate(str(start_date), str(end_date)) \\
            .select(["NDVI", "EVI"]).mean()
        ndvi = mod13.select("NDVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("NDVI").getInfo()
        evi = mod13.select("EVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("EVI").getInfo()

        # NDMI
        def add_ndmi(img):
            return img.addBands(img.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']).rename("NDMI"))

        ndmi_img = ee.ImageCollection("MODIS/061/MOD09GA") \\
            .filterBounds(point).filterDate(str(start_date), str(end_date)) \\
            .map(add_ndmi).select("NDMI").mean()
        ndmi = ndmi_img.reduceRegion(ee.Reducer.mean(), point, 500).get("NDMI").getInfo()

        # Precipitation
        precip = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \\
            .filterDate(str(start_date), str(end_date)) \\
            .select("precipitation").sum().reduceRegion(ee.Reducer.mean(), point, 5000).get("precipitation").getInfo()

        # LST
        lst_img = ee.ImageCollection("MODIS/061/MOD11A1") \\
            .filterDate(str(start_date), str(end_date)) \\
            .select("LST_Day_1km").mean()
        lst = lst_img.multiply(0.02).subtract(273.15).reduceRegion(ee.Reducer.mean(), point, 1000).get("LST_Day_1km").getInfo()

        # Treecover
        tree_img = ee.ImageCollection("MODIS/061/MOD44B") \\
            .filterBounds(point).select("Percent_Tree_Cover").mean()
        treecover = tree_img.reduceRegion(ee.Reducer.mean(), point, 250).get("Percent_Tree_Cover").getInfo()

        lst_anomaly = lst - lst_season_mean
        precip_anomaly = precip - precip_season_mean

        return [evi, ndmi, lst_anomaly, precip_anomaly, treecover]
    except:
        return [None]*5

st.info("⏳ Running prediction for first 10 grid cells...")

features_list = []
for _, row in gdf.iterrows():
    lon, lat = row.geometry.centroid.x, row.geometry.centroid.y
    feats = compute_features(lon, lat)
    if None not in feats:
        features_list.append(feats)
    else:
        features_list.append([0]*5)  # fallback or skip

X_scaled = scaler.transform(np.array(features_list))
predictions = model.predict(X_scaled)
gdf["predicted"] = predictions

enable_colors = st.checkbox("🟩🟥 Show prediction color (green: forest, red: deforested)", value=True)

m = folium.Map(location=[19.5, 74], zoom_start=7.5, control_scale=True)

for _, row in gdf.iterrows():
    pred = row["predicted"]
    fill = "gray"
    if enable_colors:
        fill = "green" if pred == 0 else "red"

    folium.GeoJson(
        row["geometry"],
        style_function=lambda x, fill_color=fill: {
            "fillColor": fill_color,
            "color": "black",
            "weight": 0.5,
            "fillOpacity": 0.6 if enable_colors else 0.1
        }
    ).add_to(m)

st.markdown("### 🗺️ Deforestation Prediction Map (First 10 Grids)")
st_folium(m, height=600, width=900)
'''


In [13]:
with open("app2.py", "w", encoding="utf-8") as f:
    f.write(code2)


In [14]:
!wget -q -O - ipv4.icanhazip.com

35.237.115.225


In [15]:
!streamlit run app2.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.115.225:8501

⠴⠦⠧your url is: https://social-ears-train.loca.lt
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
  Stoppin

In [16]:
code3 = '''
import joblib
import streamlit as st
import folium
from streamlit_folium import st_folium
from folium.plugins import Draw
import numpy as np
import geopandas as gpd
import datetime
import ee
from shapely.geometry import shape

# Initialize Earth Engine
ee.Initialize(project='ringed-trail-454308-d2')

model = joblib.load("this_ensemble_model.pkl")
scaler = joblib.load("this_scaler.pkl")

st.set_page_config(layout="wide")
st.title("🧠 Deforestation Prediction in Selected Region")

@st.cache_data
def load_grid():
    return gpd.read_file("Maharashtra_5x5km_Grid.geojson")

gdf = load_grid()
gdf["predicted"] = None

st.info("✏️ Use the draw tool to select a region on the map")

m = folium.Map(location=[19.5, 74], zoom_start=7.5, control_scale=True)
Draw(export=True).add_to(m)
map_data = st_folium(m, height=600, width=900)

if map_data.get("last_active_drawing"):
    drawn_geojson = map_data["last_active_drawing"]["geometry"]
    drawn_shape = shape(drawn_geojson)
    selected_gdf = gdf[gdf.intersects(drawn_shape)]

    if selected_gdf.empty:
        st.warning("⚠️ No grid cells found in selected region.")
    else:
        st.success(f"📦 Selected {len(selected_gdf)} grid cells")

        # Dates
        end_date = datetime.datetime.now().date()
        start_date = end_date - datetime.timedelta(days=30)
        lst_season_mean = 30
        precip_season_mean = 50

        def compute_features(lon, lat):
            point = ee.Geometry.Point([lon, lat])
            try:
                mod13 = ee.ImageCollection("MODIS/061/MOD13Q1") \\
                    .filterBounds(point).filterDate(str(start_date), str(end_date)) \\
                    .select(["NDVI", "EVI"]).mean()
                ndvi = mod13.select("NDVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("NDVI").getInfo()
                evi = mod13.select("EVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("EVI").getInfo()

                def add_ndmi(img):
                    return img.addBands(img.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']).rename("NDMI"))

                ndmi_img = ee.ImageCollection("MODIS/061/MOD09GA") \\
                    .filterBounds(point).filterDate(str(start_date), str(end_date)) \\
                    .map(add_ndmi).select("NDMI").mean()
                ndmi = ndmi_img.reduceRegion(ee.Reducer.mean(), point, 500).get("NDMI").getInfo()

                precip = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \\
                    .filterDate(str(start_date), str(end_date)) \\
                    .select("precipitation").sum().reduceRegion(ee.Reducer.mean(), point, 5000).get("precipitation").getInfo()

                lst_img = ee.ImageCollection("MODIS/061/MOD11A1") \\
                    .filterDate(str(start_date), str(end_date)) \\
                    .select("LST_Day_1km").mean()
                lst = lst_img.multiply(0.02).subtract(273.15).reduceRegion(ee.Reducer.mean(), point, 1000).get("LST_Day_1km").getInfo()

                tree_img = ee.ImageCollection("MODIS/061/MOD44B") \\
                    .filterBounds(point).select("Percent_Tree_Cover").mean()
                treecover = tree_img.reduceRegion(ee.Reducer.mean(), point, 250).get("Percent_Tree_Cover").getInfo()

                lst_anomaly = lst - lst_season_mean
                precip_anomaly = precip - precip_season_mean

                return [evi, ndmi, lst_anomaly, precip_anomaly, treecover]
            except:
                return [None]*5

        features_list = []
        for _, row in selected_gdf.iterrows():
            lon, lat = row.geometry.centroid.x, row.geometry.centroid.y
            feats = compute_features(lon, lat)
            if None not in feats:
                features_list.append(feats)
            else:
                features_list.append([0]*5)

        X_scaled = scaler.transform(np.array(features_list))
        selected_gdf["predicted"] = model.predict(X_scaled)

        # Show map with colored grids
        result_map = folium.Map(location=[19.5, 74], zoom_start=7.5, control_scale=True)
        for _, row in selected_gdf.iterrows():
            pred = row["predicted"]
            fill = "green" if pred == 0 else "red"
            folium.GeoJson(
                row["geometry"],
                style_function=lambda x, fill_color=fill: {
                    "fillColor": fill_color,
                    "color": "black",
                    "weight": 0.5,
                    "fillOpacity": 0.6
                }
            ).add_to(result_map)

        st.markdown("### 🗺️ Predicted Deforestation Map for Selected Region")
        st_folium(result_map, height=600, width=900)
else:
    st.warning("🖱️ Please draw a polygon on the map to select a region.")
'''


In [17]:
with open("app3.py", "w", encoding="utf-8") as f:
    f.write(code3)


In [20]:
!wget -q -O - ipv4.icanhazip.com

35.237.115.225


In [21]:
!streamlit run app3.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.115.225:8501

⠴⠦your url is: https://rude-mails-sniff.loca.lt
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.war

In [34]:
code4 = '''
import joblib
import streamlit as st
import folium
from streamlit_folium import st_folium
from folium.plugins import Draw
import numpy as np
import geopandas as gpd
import datetime
import ee
from shapely.geometry import shape

# Initialize Earth Engine
ee.Initialize(project='ringed-trail-454308-d2')

model = joblib.load("this_ensemble_model.pkl")
scaler = joblib.load("this_scaler.pkl")

st.set_page_config(layout="wide")
st.title("🧠 Deforestation Prediction in Selected Region")

@st.cache_data
def load_grid():
    return gpd.read_file("Maharashtra_5x5km_Grid.geojson")

gdf = load_grid()
gdf["predicted"] = None

opacity = st.slider("🔁 Set grid fill opacity", 0.1, 1.0, 0.6, step=0.1)

# Reset button
if "last_active_drawing" in st.session_state:
    if st.button("🔄 Reset and Select New Region"):
        st.session_state.pop("last_active_drawing")
        st.rerun()

# Show draw map if no selection
if "last_active_drawing" not in st.session_state:
    st.info("✏️ Use the draw tool to select a region on the map")

    m = folium.Map(location=[19.5, 74], zoom_start=7.5, control_scale=True)
    Draw(export=True).add_to(m)
    map_data = st_folium(m, height=600, width=900)

    if map_data.get("last_active_drawing"):
        st.session_state["last_active_drawing"] = map_data["last_active_drawing"]
        st.rerun()
else:
    drawn_geojson = st.session_state["last_active_drawing"]["geometry"]
    drawn_shape = shape(drawn_geojson)
    selected_gdf = gdf[gdf.intersects(drawn_shape)].copy()

    if selected_gdf.empty:
        st.warning("⚠️ No grid cells found in selected region.")
    else:
        st.success(f"📦 Selected {len(selected_gdf)} grid cells")

        # Dates
        end_date = datetime.datetime.now().date()
        start_date = end_date - datetime.timedelta(days=30)
        lst_season_mean = 30
        precip_season_mean = 50

        def compute_features(lon, lat):
            point = ee.Geometry.Point([lon, lat])
            try:
                mod13 = ee.ImageCollection("MODIS/061/MOD13Q1") \
                    .filterBounds(point).filterDate(str(start_date), str(end_date)) \
                    .select(["NDVI", "EVI"]).mean()
                ndvi = mod13.select("NDVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("NDVI").getInfo()
                evi = mod13.select("EVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("EVI").getInfo()

                def add_ndmi(img):
                    return img.addBands(img.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']).rename("NDMI"))

                ndmi_img = ee.ImageCollection("MODIS/061/MOD09GA") \
                    .filterBounds(point).filterDate(str(start_date), str(end_date)) \
                    .map(add_ndmi).select("NDMI").mean()
                ndmi = ndmi_img.reduceRegion(ee.Reducer.mean(), point, 500).get("NDMI").getInfo()

                precip = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
                    .filterDate(str(start_date), str(end_date)) \
                    .select("precipitation").sum().reduceRegion(ee.Reducer.mean(), point, 5000).get("precipitation").getInfo()

                lst_img = ee.ImageCollection("MODIS/061/MOD11A1") \
                    .filterDate(str(start_date), str(end_date)) \
                    .select("LST_Day_1km").mean()
                lst = lst_img.multiply(0.02).subtract(273.15).reduceRegion(ee.Reducer.mean(), point, 1000).get("LST_Day_1km").getInfo()

                tree_img = ee.ImageCollection("MODIS/061/MOD44B") \
                    .filterBounds(point).select("Percent_Tree_Cover").mean()
                treecover = tree_img.reduceRegion(ee.Reducer.mean(), point, 250).get("Percent_Tree_Cover").getInfo()

                lst_anomaly = lst - lst_season_mean
                precip_anomaly = precip - precip_season_mean

                return [evi, ndmi, lst_anomaly, precip_anomaly, treecover, ndvi]
            except:
                return [None]*6

        features_list = []
        param_data = []
        valid_rows = []

        for _, row in selected_gdf.iterrows():
            lon, lat = row.geometry.centroid.x, row.geometry.centroid.y
            feats = compute_features(lon, lat)
            if None not in feats:
                evi, ndmi, lst_anom, precip_anom, treecover, ndvi = feats
                features_list.append([evi, ndmi, lst_anom, precip_anom, treecover])
                param_data.append({
                    "NDVI": round(ndvi, 4),
                    "EVI": round(evi, 4),
                    "NDMI": round(ndmi, 4),
                    "LST Anomaly": round(lst_anom, 2),
                    "Precip Anomaly": round(precip_anom, 2),
                    "Treecover": round(treecover, 1)
                })
                valid_rows.append(row)

        if not features_list:
            st.error("❌ No valid data points found in selected region.")
        else:
            valid_gdf = gpd.GeoDataFrame(valid_rows).reset_index(drop=True)
            X_scaled = scaler.transform(np.array(features_list))
            valid_gdf["predicted"] = model.predict(X_scaled)

            center_lat = valid_gdf.geometry.centroid.y.mean()
            center_lon = valid_gdf.geometry.centroid.x.mean()

            result_map = folium.Map(location=[center_lat, center_lon], zoom_start=9.5, control_scale=True)

            for idx, row in valid_gdf.iterrows():
                pred = row["predicted"]
                tooltip_text = "<br>".join([f"{k}: {v}" for k, v in param_data[idx].items()])
                tooltip_text += "<br>Prediction: " + ("Deforested" if pred == 1 else "Forest")
                fill = "green" if pred == 0 else "red"

                folium.GeoJson(
                    row["geometry"],
                    tooltip=tooltip_text,
                    style_function=lambda x, fill_color=fill: {
                        "fillColor": fill_color,
                        "color": "black",
                        "weight": 0.5,
                        "fillOpacity": opacity
                    }
                ).add_to(result_map)

            st.markdown("### 🗺️ Prediction Map for Selected Region")
            st_folium(result_map, height=600, width=900)

'''

In [35]:
with open("app4.py", "w", encoding="utf-8") as f:
    f.write(code4)

In [36]:
!wget -q -O - ipv4.icanhazip.com

35.237.115.225


In [37]:
!streamlit run app4.py & npx localtunnel --port 8501

⠙

⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.115.225:8501

⠦your url is: https://honest-jars-relax.loca.lt
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
  Stopping

In [38]:
code5 = '''
# app.py
import streamlit as st
import folium
from streamlit_folium import st_folium
from folium.plugins import Draw
import geopandas as gpd
import datetime
import numpy as np
import joblib
import ee
from shapely.geometry import shape
import matplotlib.pyplot as plt
import pandas as pd

# Initialize EE
ee.Initialize(project='ringed-trail-454308-d2')

# Load model and scaler
model = joblib.load("this_ensemble_model.pkl")
scaler = joblib.load("this_scaler.pkl")

# Load grid
@st.cache_data
def load_grid():
    return gpd.read_file("Maharashtra_5x5km_Grid.geojson")

gdf = load_grid()

# Streamlit setup
st.set_page_config(layout="wide")
st.title("🌳 Deforestation Analysis Tool")

page = st.sidebar.radio("📂 Navigation", [
    "1️⃣ Visualize Environmental Layers",
    "2️⃣ Predict Forest Cover",
    "3️⃣ Timeline of Tree Cover",
    "4️⃣ Environmental Statistics"
])

# Reset feature
if "last_active_drawing" in st.session_state:
    if st.sidebar.button("🔄 Reset Region"):
        st.session_state.pop("last_active_drawing")
        st.rerun()

# Drawing map (common for all pages)
if "last_active_drawing" not in st.session_state:
    st.info("✏️ Draw a polygon to select a region")

    m = folium.Map(location=[19.5, 74], zoom_start=7.5)
    Draw(export=True).add_to(m)
    map_data = st_folium(m, height=600, width=900)

    if map_data.get("last_active_drawing"):
        st.session_state["last_active_drawing"] = map_data["last_active_drawing"]
        st.rerun()
else:
    # Selection geometry
    drawn_geojson = st.session_state["last_active_drawing"]["geometry"]
    drawn_shape = shape(drawn_geojson)
    selected_gdf = gdf[gdf.intersects(drawn_shape)].copy()

    if selected_gdf.empty:
        st.warning("⚠️ No grid found in selected region.")
    else:
        st.success(f"📍 Selected {len(selected_gdf)} grid cells")

        # Shared function to compute environmental features
        def compute_features(lon, lat, start_date, end_date):
            point = ee.Geometry.Point([lon, lat])
            try:
                mod13 = ee.ImageCollection("MODIS/061/MOD13Q1") \
                    .filterDate(str(start_date), str(end_date)) \
                    .filterBounds(point) \
                    .select(["NDVI", "EVI"]).mean()

                evi = mod13.select("EVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("EVI").getInfo()
                ndvi = mod13.select("NDVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("NDVI").getInfo()

                def add_ndmi(img):
                    return img.addBands(img.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']).rename("NDMI"))

                ndmi_img = ee.ImageCollection("MODIS/061/MOD09GA") \
                    .filterBounds(point).filterDate(str(start_date), str(end_date)) \
                    .map(add_ndmi).select("NDMI").mean()
                ndmi = ndmi_img.reduceRegion(ee.Reducer.mean(), point, 500).get("NDMI").getInfo()

                precip = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
                    .filterDate(str(start_date), str(end_date)) \
                    .filterBounds(point).sum().select("precipitation") \
                    .reduceRegion(ee.Reducer.mean(), point, 5000).get("precipitation").getInfo()

                lst_img = ee.ImageCollection("MODIS/061/MOD11A1") \
                    .filterDate(str(start_date), str(end_date)) \
                    .filterBounds(point).select("LST_Day_1km").mean()
                lst = lst_img.multiply(0.02).subtract(273.15).reduceRegion(ee.Reducer.mean(), point, 1000).get("LST_Day_1km").getInfo()

                tree_img = ee.ImageCollection("MODIS/061/MOD44B") \
                    .filterBounds(point).select("Percent_Tree_Cover").mean()
                treecover = tree_img.reduceRegion(ee.Reducer.mean(), point, 250).get("Percent_Tree_Cover").getInfo()

                return [evi, ndvi, ndmi, lst, precip, treecover]
            except:
                return [None]*6

        if page == "1️⃣ Visualize Environmental Layers":
            layer = st.selectbox("📌 Select layer to visualize", ["Tree Cover", "Precipitation", "Temperature (LST)"])

            # Determine image to use
            end_date = datetime.date.today()
            start_date = end_date - datetime.timedelta(days=30)
            region = ee.Geometry.Polygon([list(drawn_shape.exterior.coords)])

            if layer == "Tree Cover":
                img = ee.ImageCollection("MODIS/061/MOD44B") \
                    .filterBounds(region).select("Percent_Tree_Cover").mean()
                vis = {"min": 0, "max": 100, "palette": ['#f7fcf5','#e5f5e0','#a1d99b','#31a354','#006d2c']}
                label = "Tree Cover (%)"
            elif layer == "Precipitation":
                img = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
                    .filterDate(str(start_date), str(end_date)) \
                    .filterBounds(region).sum().select("precipitation")
                vis = {"min": 0, "max": 200, "palette": ['#f7fcf0', '#ccece6', '#66c2a4', '#238b45', '#005824']}
                label = "Precipitation (mm)"
            else:
                img = ee.ImageCollection("MODIS/061/MOD11A1") \
                    .filterDate(str(start_date), str(end_date)) \
                    .filterBounds(region).select("LST_Day_1km").mean() \
                    .multiply(0.02).subtract(273.15)
                vis = {"min": 10, "max": 45, "palette": ['#ffffb2','#fecc5c','#fd8d3c','#f03b20','#bd0026']}
                label = "LST (°C)"

            m = folium.Map(location=[19.5, 74], zoom_start=8)
            folium.raster_layers.TileLayer(
                tiles=img.visualize(**vis).getMapId()["tile_fetcher"].url_format,
                attr="EE", name=label, overlay=True, control=True
            ).add_to(m)
            folium.LayerControl().add_to(m)

            st.markdown(f"### 🗺️ {label} over selected region")
            st_folium(m, height=600, width=900)

        elif page == "2️⃣ Predict Forest Cover":
            st.markdown("### 🔍 Forest Prediction Map")

            features = []
            for _, row in selected_gdf.iterrows():
                lon, lat = row.geometry.centroid.x, row.geometry.centroid.y
                feats = compute_features(lon, lat, datetime.date.today() - datetime.timedelta(days=30), datetime.date.today())
                if None not in feats:
                    features.append((row, feats))

            if features:
                X = [f[1][[0,2,3,4,5]] for f in features]  # drop NDVI
                X_scaled = scaler.transform(X)
                preds = model.predict(X_scaled)

                m = folium.Map(location=[19.5, 74], zoom_start=8)
                for idx, (row, _) in enumerate(features):
                    fill_color = "green" if preds[idx] == 0 else "red"
                    folium.GeoJson(row.geometry, style_function=lambda x, c=fill_color: {
                        "fillColor": c, "color": "black", "weight": 1, "fillOpacity": 0.5
                    }).add_to(m)

                st_folium(m, height=600, width=900)
            else:
                st.warning("⚠️ No valid data found for prediction.")

        elif page == "3️⃣ Timeline of Tree Cover":
            st.markdown("### ⏳ Tree Cover Timeline (2016–2025)")

            years = list(range(2016, 2027))  # Try including 2026
            timeline_data = []

            for y in years:
                img = ee.ImageCollection("MODIS/061/MOD44B") \
                    .filterDate(f"{y}-01-01", f"{y}-12-31") \
                    .filterBounds(region) \
                    .select("Percent_Tree_Cover").mean()
                mean_val = img.reduceRegion(ee.Reducer.mean(), region, 250).get("Percent_Tree_Cover").getInfo()
                if mean_val:
                    timeline_data.append((y, round(mean_val, 2)))

            df = pd.DataFrame(timeline_data, columns=["Year", "Mean Tree Cover"])
            st.line_chart(df.set_index("Year"))

        elif page == "4️⃣ Environmental Statistics":
            st.markdown("### 📊 Environmental Stats (2015–2023)")

            stats_data = []
            for y in range(2015, 2024):
                start = ee.Date.fromYMD(y, 1, 1)
                end = start.advance(1, 'year')
                for param, ic, band, scale in [
                    ("NDVI", "MODIS/061/MOD13Q1", "NDVI", 250),
                    ("EVI", "MODIS/061/MOD13Q1", "EVI", 250),
                    ("LST", "MODIS/061/MOD11A1", "LST_Day_1km", 1000),
                    ("Precipitation", "UCSB-CHG/CHIRPS/DAILY", "precipitation", 5000)
                ]:
                    img = ee.ImageCollection(ic).filterDate(start, end).filterBounds(region).select(band).mean()
                    if band == "LST_Day_1km":
                        img = img.multiply(0.02).subtract(273.15)
                    elif band in ["NDVI", "EVI"]:
                        img = img.multiply(0.0001)
                    elif band == "precipitation":
                        img = ee.ImageCollection(ic).filterDate(start, end).filterBounds(region).select(band).sum()

                    val = img.reduceRegion(ee.Reducer.mean(), region, scale).get(band).getInfo()
                    if val:
                        stats_data.append({"Year": y, "Parameter": param, "Value": round(val, 3)})

            df = pd.DataFrame(stats_data)
            for param in df["Parameter"].unique():
                subset = df[df["Parameter"] == param].set_index("Year")
                st.line_chart(subset["Value"], height=250, use_container_width=True)

'''

In [39]:
with open("app5.py", "w", encoding="utf-8") as f:
    f.write(code5)

In [40]:
!wget -q -O - ipv4.icanhazip.com

35.237.115.225


In [ ]:
!streamlit run app5.py & npx localtunnel --port 8501

In [50]:
code6 = '''
import streamlit as st
import folium
from streamlit_folium import st_folium
from folium.plugins import Draw
import geopandas as gpd
import datetime
import numpy as np
import joblib
import ee
from shapely.geometry import shape
import matplotlib.pyplot as plt
import pandas as pd

# Initialize EE
ee.Initialize(project='ringed-trail-454308-d2')

# Load model and scaler
model = joblib.load("this_ensemble_model.pkl")
scaler = joblib.load("this_scaler.pkl")

# Load grid
@st.cache_data
def load_grid():
    return gpd.read_file("Maharashtra_5x5km_Grid.geojson")

gdf = load_grid()

# Streamlit setup
st.set_page_config(layout="wide")
st.title("🌳 Deforestation Analysis Tool")

# Navigation menu
page = st.sidebar.radio("📂 Navigation", [
    "🌍 Region Selection",
    "1️⃣ Visualize Environmental Layers",
    "2️⃣ Predict Forest Cover",
    "3️⃣ Timeline of Tree Cover",
    "4️⃣ Environmental Statistics"
])

# Page 0: Region Selection
if page == "🌍 Region Selection":
    st.header("✏️ Draw a Region to Analyze")
    m = folium.Map(location=[19.5, 74], zoom_start=7.5)
    Draw(export=True).add_to(m)
    map_data = st_folium(m, height=600, width=900)

    if map_data.get("last_active_drawing"):
        st.session_state["last_active_drawing"] = map_data["last_active_drawing"]
        st.success("✅ Region Selected! Now switch pages from the left sidebar.")

    st.stop()

# Reset
if st.sidebar.button("🔄 Reset Region"):
    st.session_state.pop("last_active_drawing", None)
    st.rerun()

if "last_active_drawing" not in st.session_state:
    st.warning("⚠️ Please go to 'Region Selection' and draw a region first.")
    st.stop()

# Selected region
drawn_shape = shape(st.session_state["last_active_drawing"]["geometry"])
selected_gdf = gdf[gdf.intersects(drawn_shape)].copy()
region = ee.Geometry.Polygon([list(drawn_shape.exterior.coords)])

if selected_gdf.empty:
    st.warning("⚠️ No grid found in selected region.")
    st.stop()

# ------------------ Helper: Compute EE features ------------------
def compute_features(lon, lat, start_date, end_date):
    point = ee.Geometry.Point([lon, lat])
    try:
        mod13 = ee.ImageCollection("MODIS/061/MOD13Q1") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(point) \
            .select(["NDVI", "EVI"]).mean()

        evi = mod13.select("EVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("EVI").getInfo()
        ndvi = mod13.select("NDVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("NDVI").getInfo()

        def add_ndmi(img):
            return img.addBands(img.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']).rename("NDMI"))

        ndmi_img = ee.ImageCollection("MODIS/061/MOD09GA") \
            .filterBounds(point).filterDate(str(start_date), str(end_date)) \
            .map(add_ndmi).select("NDMI").mean()
        ndmi = ndmi_img.reduceRegion(ee.Reducer.mean(), point, 500).get("NDMI").getInfo()

        precip = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(point).sum().select("precipitation") \
            .reduceRegion(ee.Reducer.mean(), point, 5000).get("precipitation").getInfo()

        lst_img = ee.ImageCollection("MODIS/061/MOD11A1") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(point).select("LST_Day_1km").mean()
        lst = lst_img.multiply(0.02).subtract(273.15).reduceRegion(ee.Reducer.mean(), point, 1000).get("LST_Day_1km").getInfo()

        tree_img = ee.ImageCollection("MODIS/061/MOD44B") \
            .filterBounds(point).select("Percent_Tree_Cover").mean()
        treecover = tree_img.reduceRegion(ee.Reducer.mean(), point, 250).get("Percent_Tree_Cover").getInfo()

        return [evi, ndvi, ndmi, lst, precip, treecover]
    except:
        return [None]*6

# ------------------ Page 1 ------------------
if page == "1️⃣ Visualize Environmental Layers":
    st.header("🌍 Environmental Layers Viewer")
    layer = st.selectbox("Select parameter to view", [
        "Tree Cover", "Precipitation", "LST (Temp)", "NDVI", "EVI", "NDMI"
    ])
    opacity = st.slider("🔁 Layer Opacity", 0.1, 1.0, 0.6)

    end_date = datetime.date.today()
    start_date = end_date - datetime.timedelta(days=30)

    if layer == "Tree Cover":
        img = ee.ImageCollection("MODIS/061/MOD44B") \
            .filterBounds(region).select("Percent_Tree_Cover").mean()
        vis = {"min": 0, "max": 100, "palette": ['#f7fcf5','#e5f5e0','#a1d99b','#31a354','#006d2c']}
    elif layer == "Precipitation":
        img = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(region).sum().select("precipitation")
        vis = {"min": 0, "max": 200, "palette": ['#f7fcf0', '#ccece6', '#66c2a4', '#238b45', '#005824']}
    elif layer == "LST (Temp)":
        img = ee.ImageCollection("MODIS/061/MOD11A1") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(region).select("LST_Day_1km").mean() \
            .multiply(0.02).subtract(273.15)
        vis = {"min": 10, "max": 45, "palette": ['#ffffb2','#fecc5c','#fd8d3c','#f03b20','#bd0026']}
    elif layer == "NDVI":
        img = ee.ImageCollection("MODIS/061/MOD13Q1") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(region).select("NDVI").mean().multiply(0.0001)
        vis = {"min": 0, "max": 1, "palette": ['#f7fcf5', '#00441b']}
    elif layer == "EVI":
        img = ee.ImageCollection("MODIS/061/MOD13Q1") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(region).select("EVI").mean().multiply(0.0001)
        vis = {"min": 0, "max": 1, "palette": ['#f7fcf5', '#084081']}
    else:  # NDMI
        def add_ndmi(img):
            return img.addBands(img.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']).rename("NDMI"))
        img = ee.ImageCollection("MODIS/061/MOD09GA") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(region).map(add_ndmi).select("NDMI").mean()
        vis = {"min": -1, "max": 1, "palette": ['#67001f', '#f7f7f7', '#053061']}

    mapid = img.visualize(**vis).getMapId()
    tile_url = mapid["tile_fetcher"].url_format

    m = folium.Map(location=[19.5, 74], zoom_start=8)
    folium.raster_layers.TileLayer(tiles=tile_url, name=layer, opacity=opacity, attr="Map data © Google Earth Engine").add_to(m)
    st_folium(m, height=600, width=900)

# ------------------ Page 2 ------------------
if page == "2️⃣ Predict Forest Cover":
    st.header("🧠 Forest Cover Prediction")
    st.info(f"Selected {len(selected_gdf)} grid cells for prediction.")

    end_date = datetime.date.today()
    start_date = end_date - datetime.timedelta(days=30)

    features = []
    for _, row in selected_gdf.iterrows():
        lon, lat = row.geometry.centroid.x, row.geometry.centroid.y
        feats = compute_features(lon, lat, start_date, end_date)
        features.append((row["geometry"], feats))

    X = [[f[1][i] for i in [0, 2, 3, 4, 5]] if None not in f[1] else [0]*5 for f in features]  # Drop NDVI
    param_data = [{
        "EVI": round(f[1][0], 4) if f[1][0] is not None else "N/A",
        "NDMI": round(f[1][2], 4) if f[1][2] is not None else "N/A",
        "LST Anomaly": round(f[1][3] - 30, 2) if f[1][3] is not None else "N/A",
        "Precip Anomaly": round(f[1][4] - 50, 2) if f[1][4] is not None else "N/A",
        "Tree Cover": round(f[1][5], 1) if f[1][5] is not None else "N/A"
    } for f in features]

    X_scaled = scaler.transform(np.array(X))
    y_pred = model.predict(X_scaled)

    center_lat = selected_gdf.geometry.centroid.y.mean()
    center_lon = selected_gdf.geometry.centroid.x.mean()
    opacity = st.slider("🖌️ Prediction Fill Opacity", 0.1, 1.0, 0.6)

    result_map = folium.Map(location=[center_lat, center_lon], zoom_start=9.5)

    for idx, (geom, pred) in enumerate(zip([f[0] for f in features], y_pred)):
        tooltip_text = "<br>".join([f"{k}: {v}" for k, v in param_data[idx].items()])
        tooltip_text += "<br><b>Prediction</b>: " + ("🌳 Forest" if pred == 0 else "🪵 Deforested")
        fill = "green" if pred == 0 else "red"

        folium.GeoJson(
            geom,
            tooltip=tooltip_text,
            style_function=lambda x, fill_color=fill: {
                "fillColor": fill_color,
                "color": "black",
                "weight": 0.5,
                "fillOpacity": opacity
            }
        ).add_to(result_map)

    st.subheader("🗺️ Predicted Forest Cover")
    st_folium(result_map, height=600, width=900)


# ------------------ Page 3 ------------------
if page == "3️⃣ Timeline of Tree Cover":
    st.header("📈 Tree Cover Timeline (2016–2026)")
    years = list(range(2016, 2027))
    values = []

    for y in years:
        try:
            img = ee.ImageCollection("MODIS/061/MOD44B") \
                .filterDate(f"{y}-01-01", f"{y}-12-31") \
                .filterBounds(region) \
                .select("Percent_Tree_Cover") \
                .mean()

            val_dict = img.reduceRegion(ee.Reducer.mean(), region, 250).getInfo()
            tree_val = val_dict.get("Percent_Tree_Cover", 0)
            values.append(tree_val)
        except Exception as e:
            st.warning(f"No data for year {y} or error occurred: {e}")
            values.append(0)

    fig, ax = plt.subplots()
    ax.plot(years, values, marker='o', linestyle='-')
    ax.set_xlabel("Year")
    ax.set_ylabel("Tree Cover (%)")
    ax.set_title("Average Tree Cover Over Time")
    ax.grid(True)
    st.pyplot(fig)


# ------------------ Page 4 ------------------
if page == "4️⃣ Environmental Statistics":
    st.header("📊 Environmental Stats (2015–2023)")
    param_bands = {
        "NDVI": ("MODIS/061/MOD13Q1", "NDVI", 250, 0.0001),
        "EVI": ("MODIS/061/MOD13Q1", "EVI", 250, 0.0001),
        "Precipitation": ("UCSB-CHG/CHIRPS/DAILY", "precipitation", 5000, 1),
        "LST (°C)": ("MODIS/061/MOD11A1", "LST_Day_1km", 1000, 0.02)
    }

    start_year = 2015
    end_year = 2023
    data = {k: [] for k in param_bands}
    labels = []

    for y in range(start_year, end_year + 1):
        start = f"{y}-01-01"
        end = f"{y}-12-31"
        labels.append(str(y))
        for name, (ic, band, scale, factor) in param_bands.items():
            coll = ee.ImageCollection(ic).filterDate(start, end).filterBounds(region)
            img = coll.select(band).mean()
            if name == "LST (°C)":
                img = img.multiply(factor).subtract(273.15)
            elif factor != 1:
                img = img.multiply(factor)
            val = img.reduceRegion(ee.Reducer.mean(), region, scale).get(band).getInfo()
            data[name].append(val if val is not None else 0)

    fig, ax = plt.subplots(figsize=(10, 6))
    for name, series in data.items():
        ax.plot(labels, series, marker='o', label=name)
    ax.set_xlabel("Year")
    ax.set_title("Environmental Parameter Trends")
    ax.legend()
    ax.grid(True)
    st.pyplot(fig)

'''

In [51]:
with open("app6.py", "w", encoding="utf-8") as f:
    f.write(code6)

In [52]:
!wget -q -O - ipv4.icanhazip.com

35.237.115.225


In [53]:
!streamlit run app6.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.115.225:8501

⠼⠴your url is: https://twenty-states-relate.loca.lt
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/content/app6.py:165: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lat = selected_gdf.geometry.centroid.y.mean()
/content/app6.py:166: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lon = selected_gdf.geometry.centroid.x.mean()
/usr/local/lib/python3.11/dist-packages/sklearn/utils/

In [59]:
code7 = '''
import streamlit as st
import folium
from streamlit_folium import st_folium
from folium.plugins import Draw
import geopandas as gpd
import datetime
import numpy as np
import joblib
import ee
from shapely.geometry import shape
import matplotlib.pyplot as plt
import pandas as pd

# Initialize EE
ee.Initialize(project='ringed-trail-454308-d2')

# Load model and scaler
model = joblib.load("this_ensemble_model.pkl")
scaler = joblib.load("this_scaler.pkl")

# Load grid
@st.cache_data
def load_grid():
    return gpd.read_file("Maharashtra_5x5km_Grid.geojson")

gdf = load_grid()

# Streamlit setup
st.set_page_config(layout="wide")
st.title("🌳 Deforestation Analysis Tool")

# Navigation menu
page = st.sidebar.radio("📂 Navigation", [
    "🌍 Region Selection",
    "1️⃣ Visualize Environmental Layers",
    "2️⃣ Predict Forest Cover",
    "3️⃣ Timeline of Tree Cover",
    "4️⃣ Environmental Statistics"
])

# Page 0: Region Selection
if page == "🌍 Region Selection":
    st.header("✏️ Draw a Region to Analyze")
    m = folium.Map(location=[19.5, 74], zoom_start=7.5)
    Draw(export=True).add_to(m)
    map_data = st_folium(m, height=600, width=900)

    if map_data.get("last_active_drawing"):
        st.session_state["last_active_drawing"] = map_data["last_active_drawing"]
        st.success("✅ Region Selected! Now switch pages from the left sidebar.")

    st.stop()

# Reset
if st.sidebar.button("🔄 Reset Region"):
    st.session_state.pop("last_active_drawing", None)
    st.rerun()

if "last_active_drawing" not in st.session_state:
    st.warning("⚠️ Please go to 'Region Selection' and draw a region first.")
    st.stop()

# Selected region
drawn_shape = shape(st.session_state["last_active_drawing"]["geometry"])
selected_gdf = gdf[gdf.intersects(drawn_shape)].copy()
region = ee.Geometry.Polygon([list(drawn_shape.exterior.coords)])

if selected_gdf.empty:
    st.warning("⚠️ No grid found in selected region.")
    st.stop()

# ------------------ Helper: Compute EE features ------------------
def compute_features(lon, lat, start_date, end_date):
    point = ee.Geometry.Point([lon, lat])
    try:
        mod13 = ee.ImageCollection("MODIS/061/MOD13Q1") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(point) \
            .select(["NDVI", "EVI"]).mean()

        evi = mod13.select("EVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("EVI").getInfo()
        ndvi = mod13.select("NDVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("NDVI").getInfo()

        def add_ndmi(img):
            return img.addBands(img.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']).rename("NDMI"))

        ndmi_img = ee.ImageCollection("MODIS/061/MOD09GA") \
            .filterBounds(point).filterDate(str(start_date), str(end_date)) \
            .map(add_ndmi).select("NDMI").mean()
        ndmi = ndmi_img.reduceRegion(ee.Reducer.mean(), point, 500).get("NDMI").getInfo()

        precip = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(point).sum().select("precipitation") \
            .reduceRegion(ee.Reducer.mean(), point, 5000).get("precipitation").getInfo()

        lst_img = ee.ImageCollection("MODIS/061/MOD11A1") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(point).select("LST_Day_1km").mean()
        lst = lst_img.multiply(0.02).subtract(273.15).reduceRegion(ee.Reducer.mean(), point, 1000).get("LST_Day_1km").getInfo()

        tree_img = ee.ImageCollection("MODIS/061/MOD44B") \
            .filterBounds(point).select("Percent_Tree_Cover").mean()
        treecover = tree_img.reduceRegion(ee.Reducer.mean(), point, 250).get("Percent_Tree_Cover").getInfo()

        return [evi, ndvi, ndmi, lst, precip, treecover]
    except:
        return [None]*6

# ------------------ Page 1 ------------------
if page == "1️⃣ Visualize Environment Layers":
    st.header("🌍 Environmental Parameters in Selected Region")
    opacity = st.slider("🖌️ Layer Opacity", 0.1, 1.0, 0.6)

    vis_options = {
        "LST (°C)": {
            "collection": "MODIS/061/MOD11A1",
            "band": "LST_Day_1km",
            "scale": 1000,
            "palette": ['#f7fcf0', '#ccece6', '#66c2a4', '#238b45', '#005824'],
            "transform": lambda img: img.multiply(0.02).subtract(273.15),
            "vmin": 15, "vmax": 45
        },
        "Precipitation (mm)": {
            "collection": "UCSB-CHG/CHIRPS/DAILY",
            "band": "precipitation",
            "scale": 5000,
            "palette": ['#f7fbff', '#deebf7', '#9ecae1', '#3182bd', '#08519c'],
            "transform": lambda img: img,
            "vmin": 0, "vmax": 200
        },
        "Tree Cover (%)": {
            "collection": "MODIS/061/MOD44B",
            "band": "Percent_Tree_Cover",
            "scale": 250,
            "palette": ['#f7fcfd', '#e0ecf4', '#bfd3e6', '#9ebcda', '#8c96c6', '#8856a7', '#810f7c'],
            "transform": lambda img: img,
            "vmin": 0, "vmax": 100
        },
        "EVI": {
            "collection": "MODIS/061/MOD13Q1",
            "band": "EVI",
            "scale": 250,
            "palette": ['#ffffcc', '#c2e699', '#78c679', '#31a354', '#006837'],
            "transform": lambda img: img.multiply(0.0001),
            "vmin": 0, "vmax": 0.8
        },
        "NDMI": {
            "collection": "MODIS/061/MOD09GA",
            "band": "NDMI",
            "scale": 500,
            "palette": ['#ffffe5', '#f7fcb9', '#addd8e', '#31a354', '#006837'],
            "transform": lambda img: img.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']).rename("NDMI"),
            "vmin": -0.2, "vmax": 0.6
        }
    }

    selected_layer = st.selectbox("🌐 Choose a parameter to visualize", list(vis_options.keys()))
    vis_info = vis_options[selected_layer]

    col = ee.ImageCollection(vis_info["collection"]) \
        .filterDate(str(datetime.date.today() - datetime.timedelta(days=30)), str(datetime.date.today())) \
        .select(vis_info["band"])

    img = col.mean()
    img = vis_info["transform"](img)
    img = img.clip(region)  # ✅ Clip to selected region only

    vis_params = {
        "min": vis_info["vmin"],
        "max": vis_info["vmax"],
        "palette": vis_info["palette"]
    }

    tile_url = img.getMapId(vis_params)["tile_fetcher"].url_format
    m = folium.Map(location=[region.centroid().coordinates().getInfo()[1],
                             region.centroid().coordinates().getInfo()[0]], zoom_start=8)

    folium.raster_layers.TileLayer(
        tiles=tile_url,
        name=selected_layer,
        opacity=opacity,
        attr="Map data © Google Earth Engine"
    ).add_to(m)

    folium.GeoJson(region.getInfo(), name="Selected Region",
                   style_function=lambda x: {
                       "fillOpacity": 0,
                       "color": "black",
                       "weight": 2
                   }).add_to(m)

    st_folium(m, height=600, width=900)

# ------------------ Page 2 ------------------
if page == "2️⃣ Predict Forest Cover":
    st.header("🧠 Forest Cover Prediction")
    st.info(f"Selected {len(selected_gdf)} grid cells for prediction.")

    end_date = datetime.date.today()
    start_date = end_date - datetime.timedelta(days=30)

    features = []
    for _, row in selected_gdf.iterrows():
        lon, lat = row.geometry.centroid.x, row.geometry.centroid.y
        feats = compute_features(lon, lat, start_date, end_date)
        features.append((row["geometry"], feats))

    X = [[f[1][i] for i in [0, 2, 3, 4, 5]] if None not in f[1] else [0]*5 for f in features]  # Drop NDVI
    param_data = [{
        "EVI": round(f[1][0], 4) if f[1][0] is not None else "N/A",
        "NDMI": round(f[1][2], 4) if f[1][2] is not None else "N/A",
        "LST Anomaly": round(f[1][3] - 30, 2) if f[1][3] is not None else "N/A",
        "Precip Anomaly": round(f[1][4] - 50, 2) if f[1][4] is not None else "N/A",
        "Tree Cover": round(f[1][5], 1) if f[1][5] is not None else "N/A"
    } for f in features]

    X_scaled = scaler.transform(np.array(X))
    y_pred = model.predict(X_scaled)

    center_lat = selected_gdf.geometry.centroid.y.mean()
    center_lon = selected_gdf.geometry.centroid.x.mean()
    opacity = st.slider("🖌️ Prediction Fill Opacity", 0.1, 1.0, 0.6)

    result_map = folium.Map(location=[center_lat, center_lon], zoom_start=9.5)

    for idx, (geom, pred) in enumerate(zip([f[0] for f in features], y_pred)):
        tooltip_text = "<br>".join([f"{k}: {v}" for k, v in param_data[idx].items()])
        tooltip_text += "<br><b>Prediction</b>: " + ("🌳 Forest" if pred == 0 else "🪵 Deforested")
        fill = "green" if pred == 0 else "red"

        folium.GeoJson(
            geom,
            tooltip=tooltip_text,
            style_function=lambda x, fill_color=fill: {
                "fillColor": fill_color,
                "color": "black",
                "weight": 0.5,
                "fillOpacity": opacity
            }
        ).add_to(result_map)

    st.subheader("🗺️ Predicted Forest Cover")
    st_folium(result_map, height=600, width=900)


# ------------------ Page 3 ------------------
if page == "3️⃣ Timeline of Tree Cover":
    st.header("📽️ Animated Timeline of EVI (2016–2025)")
    years = list(range(2016, 2026))

    image_list = []

    for y in years:
        # Create EVI mean image for the year
        img = ee.ImageCollection("MODIS/061/MOD13Q1") \
            .filterDate(f"{y}-01-01", f"{y}-12-31") \
            .filterBounds(region) \
            .select("EVI") \
            .mean() \
            .multiply(0.0001) \
            .clip(region)

        # Visualize it for thumbnail export
        vis = img.visualize(**{
            "min": 0,
            "max": 0.8,
            "palette": ['#ffffcc', '#c2e699', '#78c679', '#31a354', '#006837']
        }).set({'label': str(y)})

        image_list.append(vis)

    # Convert to server-side ImageCollection
    gif_collection = ee.ImageCollection.fromImages(image_list)

    # Define video thumbnail arguments
    video_args = {
        'dimensions': 600,
        'region': region.bounds(),
        'framesPerSecond': 1,
        'format': 'gif'
    }

    # Get thumbnail (animated gif) URL
    url = gif_collection.getVideoThumbURL(video_args)

    st.markdown("#### 🌿 EVI Timeline Over Selected Region")
    st.image(url, caption="EVI Animation (2016–2025)")


# ------------------ Page 4 ------------------
if page == "4️⃣ Environmental Statistics":
    st.header("📊 Environmental Stats (2015–2023)")
    param_bands = {
        "NDVI": ("MODIS/061/MOD13Q1", "NDVI", 250, 0.0001),
        "EVI": ("MODIS/061/MOD13Q1", "EVI", 250, 0.0001),
        "Precipitation": ("UCSB-CHG/CHIRPS/DAILY", "precipitation", 5000, 1),
        "LST (°C)": ("MODIS/061/MOD11A1", "LST_Day_1km", 1000, 0.02)
    }

    start_year = 2015
    end_year = 2023
    data = {k: [] for k in param_bands}
    labels = []

    for y in range(start_year, end_year + 1):
        start = f"{y}-01-01"
        end = f"{y}-12-31"
        labels.append(str(y))
        for name, (ic, band, scale, factor) in param_bands.items():
            coll = ee.ImageCollection(ic).filterDate(start, end).filterBounds(region)
            img = coll.select(band).mean()
            if name == "LST (°C)":
                img = img.multiply(factor).subtract(273.15)
            elif factor != 1:
                img = img.multiply(factor)
            val = img.reduceRegion(ee.Reducer.mean(), region, scale).get(band).getInfo()
            data[name].append(val if val is not None else 0)

    fig, ax = plt.subplots(figsize=(10, 6))
    for name, series in data.items():
        ax.plot(labels, series, marker='o', label=name)
    ax.set_xlabel("Year")
    ax.set_title("Environmental Parameter Trends")
    ax.legend()
    ax.grid(True)
    st.pyplot(fig)

'''

In [60]:
with open("app7.py", "w", encoding="utf-8") as f:
    f.write(code7)

In [61]:
!wget -q -O - ipv4.icanhazip.com

35.237.115.225


In [62]:
!streamlit run app7.py & npx localtunnel --port 8501

⠙

⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.115.225:8501

⠴⠦your url is: https://moody-seals-rule.loca.lt
  Stopping...
^C


In [123]:
code8 = '''
import streamlit as st
import folium
from streamlit_folium import st_folium
from folium.plugins import Draw
import geopandas as gpd
import datetime
import numpy as np
import joblib
import ee
from shapely.geometry import shape
import matplotlib.pyplot as plt
import pandas as pd

# Initialize EE
ee.Initialize(project='ringed-trail-454308-d2')

# Load model and scaler
model = joblib.load("this_ensemble_model.pkl")
scaler = joblib.load("this_scaler.pkl")

# Load grid
@st.cache_data
def load_grid():
    return gpd.read_file("Maharashtra_5x5km_Grid.geojson")

gdf = load_grid()

# Streamlit setup
st.set_page_config(layout="wide")
st.title("🌳 Deforestation Analysis Tool")

# Navigation menu
page = st.sidebar.radio("📂 Navigation", [
    "🌍 Region Selection",
    "1️⃣ Visualize Environmental Layers",
    "2️⃣ Predict Forest Cover",
    "3️⃣ Timeline of Tree Cover",
    "4️⃣ Environmental Statistics"
])

# Page 0: Region Selection
if page == "🌍 Region Selection":
    st.header("✏️ Draw a Region to Analyze")
    m = folium.Map(location=[19.5, 74], zoom_start=7.5)
    Draw(export=True).add_to(m)
    map_data = st_folium(m, height=600, width=900)

    if map_data.get("last_active_drawing"):
        st.session_state["last_active_drawing"] = map_data["last_active_drawing"]
        st.success("✅ Region Selected! Now switch pages from the left sidebar.")

    st.stop()

# Reset
if st.sidebar.button("🔄 Reset Region"):
    st.session_state.pop("last_active_drawing", None)
    st.rerun()

if "last_active_drawing" not in st.session_state:
    st.warning("⚠️ Please go to 'Region Selection' and draw a region first.")
    st.stop()

# Selected region
drawn_shape = shape(st.session_state["last_active_drawing"]["geometry"])
selected_gdf = gdf[gdf.intersects(drawn_shape)].copy()
region = ee.Geometry.Polygon([list(drawn_shape.exterior.coords)])

if selected_gdf.empty:
    st.warning("⚠️ No grid found in selected region.")
    st.stop()

# ------------------ Helper: Compute EE features ------------------
def compute_features(lon, lat, start_date, end_date):
    point = ee.Geometry.Point([lon, lat])
    try:
        mod13 = ee.ImageCollection("MODIS/061/MOD13Q1") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(point) \
            .select(["NDVI", "EVI"]).mean()

        evi = mod13.select("EVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("EVI").getInfo()
        ndvi = mod13.select("NDVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("NDVI").getInfo()

        def add_ndmi(img):
            return img.addBands(img.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']).rename("NDMI"))

        ndmi_img = ee.ImageCollection("MODIS/061/MOD09GA") \
            .filterBounds(point).filterDate(str(start_date), str(end_date)) \
            .map(add_ndmi).select("NDMI").mean()
        ndmi = ndmi_img.reduceRegion(ee.Reducer.mean(), point, 500).get("NDMI").getInfo()

        precip = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(point).sum().select("precipitation") \
            .reduceRegion(ee.Reducer.mean(), point, 5000).get("precipitation").getInfo()

        lst_img = ee.ImageCollection("MODIS/061/MOD11A1") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(point).select("LST_Day_1km").mean()
        lst = lst_img.multiply(0.02).subtract(273.15).reduceRegion(ee.Reducer.mean(), point, 1000).get("LST_Day_1km").getInfo()

        tree_img = ee.ImageCollection("MODIS/061/MOD44B") \
            .filterBounds(point).select("Percent_Tree_Cover").mean()
        treecover = tree_img.reduceRegion(ee.Reducer.mean(), point, 250).get("Percent_Tree_Cover").getInfo()

        return [evi, ndvi, ndmi, lst, precip, treecover]
    except:
        return [None]*6

# ------------------ Page 1 ------------------
if page == "1️⃣ Visualize Environment Layers":
    st.header("🌍 Environmental Parameters in Selected Region")
    opacity = st.slider("🖌️ Layer Opacity", 0.1, 1.0, 0.6)

    vis_options = {
        "LST (°C)": {
            "collection": "MODIS/061/MOD11A1",
            "band": "LST_Day_1km",
            "scale": 1000,
            "palette": ['#f7fcf0', '#ccece6', '#66c2a4', '#238b45', '#005824'],
            "transform": lambda img: img.multiply(0.02).subtract(273.15),
            "vmin": 15, "vmax": 45
        },
        "Precipitation (mm)": {
            "collection": "UCSB-CHG/CHIRPS/DAILY",
            "band": "precipitation",
            "scale": 5000,
            "palette": ['#f7fbff', '#deebf7', '#9ecae1', '#3182bd', '#08519c'],
            "transform": lambda img: img,
            "vmin": 0, "vmax": 200
        },
        "Tree Cover (%)": {
            "collection": "MODIS/061/MOD44B",
            "band": "Percent_Tree_Cover",
            "scale": 250,
            "palette": ['#f7fcfd', '#e0ecf4', '#bfd3e6', '#9ebcda', '#8c96c6', '#8856a7', '#810f7c'],
            "transform": lambda img: img,
            "vmin": 0, "vmax": 100
        },
        "EVI": {
            "collection": "MODIS/061/MOD13Q1",
            "band": "EVI",
            "scale": 250,
            "palette": ['#ffffcc', '#c2e699', '#78c679', '#31a354', '#006837'],
            "transform": lambda img: img.multiply(0.0001),
            "vmin": 0, "vmax": 0.8
        },
        "NDMI": {
            "collection": "MODIS/061/MOD09GA",
            "band": "NDMI",
            "scale": 500,
            "palette": ['#ffffe5', '#f7fcb9', '#addd8e', '#31a354', '#006837'],
            "transform": lambda img: img.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']).rename("NDMI"),
            "vmin": -0.2, "vmax": 0.6
        }
    }

    selected_layer = st.selectbox("🌐 Choose a parameter to visualize", list(vis_options.keys()))
    vis_info = vis_options[selected_layer]

    col = ee.ImageCollection(vis_info["collection"]) \
        .filterDate(str(datetime.date.today() - datetime.timedelta(days=30)), str(datetime.date.today())) \
        .select(vis_info["band"])

    img = col.mean()
    img = vis_info["transform"](img)
    img = img.clip(region)  # ✅ Clip to selected region only

    vis_params = {
        "min": vis_info["vmin"],
        "max": vis_info["vmax"],
        "palette": vis_info["palette"]
    }

    tile_url = img.getMapId(vis_params)["tile_fetcher"].url_format
    m = folium.Map(location=[region.centroid().coordinates().getInfo()[1],
                             region.centroid().coordinates().getInfo()[0]], zoom_start=8)

    folium.raster_layers.TileLayer(
        tiles=tile_url,
        name=selected_layer,
        opacity=opacity,
        attr="Map data © Google Earth Engine"
    ).add_to(m)

    folium.GeoJson(region.getInfo(), name="Selected Region",
                   style_function=lambda x: {
                       "fillOpacity": 0,
                       "color": "black",
                       "weight": 2
                   }).add_to(m)

    st_folium(m, height=600, width=900)

# ------------------ Page 2 ------------------
if page == "2️⃣ Predict Forest Cover":
    st.header("🧠 Forest Cover Prediction")
    st.info(f"Selected {len(selected_gdf)} grid cells for prediction.")

    end_date = datetime.date.today()
    start_date = end_date - datetime.timedelta(days=30)

    features = []
    for _, row in selected_gdf.iterrows():
        lon, lat = row.geometry.centroid.x, row.geometry.centroid.y
        feats = compute_features(lon, lat, start_date, end_date)
        features.append((row["geometry"], feats))

    X = [[f[1][i] for i in [0, 2, 3, 4, 5]] if None not in f[1] else [0]*5 for f in features]  # Drop NDVI
    param_data = [{
        "EVI": round(f[1][0], 4) if f[1][0] is not None else "N/A",
        "NDMI": round(f[1][2], 4) if f[1][2] is not None else "N/A",
        "LST Anomaly": round(f[1][3] - 30, 2) if f[1][3] is not None else "N/A",
        "Precip Anomaly": round(f[1][4] - 50, 2) if f[1][4] is not None else "N/A",
        "Tree Cover": round(f[1][5], 1) if f[1][5] is not None else "N/A"
    } for f in features]

    X_scaled = scaler.transform(np.array(X))
    y_pred = model.predict(X_scaled)

    center_lat = selected_gdf.geometry.centroid.y.mean()
    center_lon = selected_gdf.geometry.centroid.x.mean()
    opacity = st.slider("🖌️ Prediction Fill Opacity", 0.1, 1.0, 0.6)

    result_map = folium.Map(location=[center_lat, center_lon], zoom_start=9.5)

    for idx, (geom, pred) in enumerate(zip([f[0] for f in features], y_pred)):
        tooltip_text = "<br>".join([f"{k}: {v}" for k, v in param_data[idx].items()])
        tooltip_text += "<br><b>Prediction</b>: " + ("🌳 Forest" if pred == 0 else "🪵 Deforested")
        fill = "green" if pred == 0 else "red"

        folium.GeoJson(
            geom,
            tooltip=tooltip_text,
            style_function=lambda x, fill_color=fill: {
                "fillColor": fill_color,
                "color": "black",
                "weight": 0.5,
                "fillOpacity": opacity
            }
        ).add_to(result_map)

    st.subheader("🗺️ Predicted Forest Cover")
    st_folium(result_map, height=600, width=900)


# ------------------ Page 3 ------------------
if page == "3️⃣ Timeline of Tree Cover":
    st.header("📽️ Animated Timeline of EVI (2016–2025)")
    years = list(range(2016, 2026))

    # --------- Helper function to create EVI image ---------
    def create_evi_image(year):
        evi = ee.ImageCollection("MODIS/061/MOD13Q1") \
            .filterDate(f"{year}-01-01", f"{year}-12-31") \
            .filterBounds(region) \
            .select("EVI") \
            .mean() \
            .multiply(0.0001) \
            .clip(region)

        evi_viz = evi.visualize(**{
            "min": 0,
            "max": 0.8,
            "palette": ['#ffffcc', '#c2e699', '#78c679', '#31a354', '#006837']
        })

        return evi_viz.set({'label': str(year)})

    # Generate image list for GIF
    image_list = [create_evi_image(y) for y in years]
    gif_collection = ee.ImageCollection(image_list)

    video_args = {
        'dimensions': 400,  # Smaller gif width
        'region': region.bounds(),
        'framesPerSecond': 1,
        'format': 'gif'
    }

    gif_url = gif_collection.getVideoThumbURL(video_args)

    st.subheader("🎞️ EVI Animation Over Selected Region")
    st.image(gif_url, caption="🌿 Animated EVI Change (2016–2025)", width=400)

    # --------- Strip of yearly EVI thumbnails ----------
    st.markdown("### 📊 Yearly EVI Snapshots")

    thumb_images = []
    for year in years:
        evi = ee.ImageCollection("MODIS/061/MOD13Q1") \
            .filterDate(f"{year}-01-01", f"{year}-12-31") \
            .filterBounds(region) \
            .select("EVI") \
            .mean() \
            .multiply(0.0001) \
            .clip(region)

        vis = {
            "min": 0,
            "max": 0.8,
            "palette": ['#ffffcc', '#c2e699', '#78c679', '#31a354', '#006837']
        }

        url = evi.visualize(**vis).getThumbURL({
            "region": region.bounds().getInfo(),
            "dimensions": 150,
            "format": "png"
        })

        thumb_images.append((year, url))

    # Display as a strip
    cols = st.columns(len(thumb_images))
    for i, (yr, url) in enumerate(thumb_images):
        with cols[i]:
            st.image(url, caption=str(yr), use_container_width=True)


# ------------------ Page 4 ------------------
if page == "4️⃣ Environmental Statistics":
    st.header("📊 Environmental Stats (2015–2023)")
    param_bands = {
        "NDVI": ("MODIS/061/MOD13Q1", "NDVI", 250, 0.0001),
        "EVI": ("MODIS/061/MOD13Q1", "EVI", 250, 0.0001),
        "Precipitation": ("UCSB-CHG/CHIRPS/DAILY", "precipitation", 5000, 1),
        "LST (°C)": ("MODIS/061/MOD11A1", "LST_Day_1km", 1000, 0.02)
    }

    start_year = 2015
    end_year = 2023
    data = {k: [] for k in param_bands}
    labels = []

    for y in range(start_year, end_year + 1):
        start = f"{y}-01-01"
        end = f"{y}-12-31"
        labels.append(str(y))
        for name, (ic, band, scale, factor) in param_bands.items():
            coll = ee.ImageCollection(ic).filterDate(start, end).filterBounds(region)
            img = coll.select(band).mean()
            if name == "LST (°C)":
                img = img.multiply(factor).subtract(273.15)
            elif factor != 1:
                img = img.multiply(factor)
            val = img.reduceRegion(ee.Reducer.mean(), region, scale).get(band).getInfo()
            data[name].append(val if val is not None else 0)

    fig, ax = plt.subplots(figsize=(10, 6))
    for name, series in data.items():
        ax.plot(labels, series, marker='o', label=name)
    ax.set_xlabel("Year")
    ax.set_title("Environmental Parameter Trends")
    ax.legend()
    ax.grid(True)
    st.pyplot(fig)

'''

In [124]:
with open("app8.py", "w", encoding="utf-8") as f:
    f.write(code8)

In [125]:
!wget -q -O - ipv4.icanhazip.com

35.237.115.225


In [126]:
!streamlit run app8.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.115.225:8501

⠼⠴your url is: https://hungry-keys-strive.loca.lt
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/content/app8.py:209: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lat = selected_gdf.geometry.centroid.y.mean()
/content/app8.py:210: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lon = selected_gdf.geometry.centroid.x.mean()
/usr/local/lib/python3.11/dist-packages/sklearn/utils/va

In [131]:
code9 = '''
import streamlit as st
import folium
from streamlit_folium import st_folium
from folium.plugins import Draw
import geopandas as gpd
import datetime
import numpy as np
import joblib
import ee
from shapely.geometry import shape
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Initialize EE
ee.Initialize(project='ringed-trail-454308-d2')

# Load model and scaler
model = joblib.load("this_ensemble_model.pkl")
scaler = joblib.load("this_scaler.pkl")

# Load grid
@st.cache_data
def load_grid():
    return gpd.read_file("Maharashtra_5x5km_Grid.geojson")

gdf = load_grid()

# Streamlit setup
st.set_page_config(layout="wide")
st.title("🌳 Deforestation Analysis Tool")

# Navigation menu
page = st.sidebar.radio("📂 Navigation", [
    "Region Selection",
    "Predict Forest Cover",
    "Timeline of Tree Cover",
    "Environmental Statistics"
])

# Page 0: Region Selection
if page == "Region Selection":
    st.header(":pencil2: Draw a Region to Analyze")
    m = folium.Map(location=[19.5, 74], zoom_start=7.5)
    Draw(export=True).add_to(m)
    map_data = st_folium(m, height=600, width=900)

    if map_data.get("last_active_drawing"):
        st.session_state["last_active_drawing"] = map_data["last_active_drawing"]
        st.success("✅ Region Selected! Now switch pages from the left sidebar.")

    st.stop()

# Reset button
if st.sidebar.button("🔄 Reset Region"):
    st.session_state.pop("last_active_drawing", None)
    st.rerun()

if "last_active_drawing" not in st.session_state:
    st.warning("⚠️ Please go to 'Region Selection' and draw a region first.")
    st.stop()

# Selected region
drawn_shape = shape(st.session_state["last_active_drawing"]["geometry"])
selected_gdf = gdf[gdf.intersects(drawn_shape)].copy()
region = ee.Geometry.Polygon([list(drawn_shape.exterior.coords)])

if selected_gdf.empty:
    st.warning("⚠️ No grid found in selected region.")
    st.stop()

# ------------------ Helper: Compute EE features ------------------
def compute_features(lon, lat, start_date, end_date):
    point = ee.Geometry.Point([lon, lat])
    try:
        mod13 = ee.ImageCollection("MODIS/061/MOD13Q1") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(point) \
            .select(["NDVI", "EVI"]).mean()

        evi = mod13.select("EVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("EVI").getInfo()
        ndvi = mod13.select("NDVI").multiply(0.0001).reduceRegion(ee.Reducer.mean(), point, 250).get("NDVI").getInfo()

        def add_ndmi(img):
            return img.addBands(img.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']).rename("NDMI"))

        ndmi_img = ee.ImageCollection("MODIS/061/MOD09GA") \
            .filterBounds(point).filterDate(str(start_date), str(end_date)) \
            .map(add_ndmi).select("NDMI").mean()
        ndmi = ndmi_img.reduceRegion(ee.Reducer.mean(), point, 500).get("NDMI").getInfo()

        precip = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(point).sum().select("precipitation") \
            .reduceRegion(ee.Reducer.mean(), point, 5000).get("precipitation").getInfo()

        lst_img = ee.ImageCollection("MODIS/061/MOD11A1") \
            .filterDate(str(start_date), str(end_date)) \
            .filterBounds(point).select("LST_Day_1km").mean()
        lst = lst_img.multiply(0.02).subtract(273.15).reduceRegion(ee.Reducer.mean(), point, 1000).get("LST_Day_1km").getInfo()

        tree_img = ee.ImageCollection("MODIS/061/MOD44B") \
            .filterBounds(point).select("Percent_Tree_Cover").mean()
        treecover = tree_img.reduceRegion(ee.Reducer.mean(), point, 250).get("Percent_Tree_Cover").getInfo()

        return [evi, ndvi, ndmi, lst, precip, treecover]
    except:
        return [None]*6

# ------------------ Page 1 ------------------
if page == "Predict Forest Cover":
    st.header("🧐 Forest Cover Prediction")
    st.info(f"Selected {len(selected_gdf)} grid cells for prediction.")

    end_date = datetime.date.today()
    start_date = end_date - datetime.timedelta(days=30)

    features = []
    for _, row in selected_gdf.iterrows():
        lon, lat = row.geometry.centroid.x, row.geometry.centroid.y
        feats = compute_features(lon, lat, start_date, end_date)
        features.append((row["geometry"], feats))

    X = [[f[1][i] for i in [0, 2, 3, 4, 5]] if None not in f[1] else [0]*5 for f in features]
    X_scaled = scaler.transform(np.array(X))
    y_pred = model.predict(X_scaled)

    param_data = [{
        "EVI": round(f[1][0], 4) if f[1][0] is not None else "N/A",
        "NDMI": round(f[1][2], 4) if f[1][2] is not None else "N/A",
        "LST": round(f[1][3], 2) if f[1][3] is not None else "N/A",
        "Precip": round(f[1][4], 2) if f[1][4] is not None else "N/A",
        "Tree Cover": round(f[1][5], 1) if f[1][5] is not None else "N/A"
    } for f in features]

    result_map = folium.Map(location=[region.centroid().coordinates().getInfo()[1],
                                      region.centroid().coordinates().getInfo()[0]], zoom_start=9.5)

    for idx, (geom, pred) in enumerate(zip([f[0] for f in features], y_pred)):
        tooltip_text = "<br>".join([f"{k}: {v}" for k, v in param_data[idx].items()])
        tooltip_text += "<br><b>Prediction</b>: " + ("Forest" if pred == 0 else "Deforested")
        fill = "green" if pred == 0 else "red"

        folium.GeoJson(
            geom,
            tooltip=tooltip_text,
            style_function=lambda x, fill_color=fill: {
                "fillColor": fill_color,
                "color": "black",
                "weight": 0.5,
                "fillOpacity": 0.6
            }
        ).add_to(result_map)

    st.subheader("🗺️ Predicted Forest Cover")
    st_folium(result_map, height=600, width=900)

# ------------------ Page 2 ------------------
if page == "Timeline of Tree Cover":
    st.header("📽️ Animated Timeline of EVI (2016–2025)")
    years = list(range(2016, 2026))

    def create_evi_image(year):
        evi = ee.ImageCollection("MODIS/061/MOD13Q1") \
            .filterDate(f"{year}-01-01", f"{year}-12-31") \
            .filterBounds(region) \
            .select("EVI") \
            .mean() \
            .multiply(0.0001) \
            .clip(region)

        return evi.visualize(min=0, max=0.8, palette=['#ffffcc', '#c2e699', '#78c679', '#31a354', '#006837']).set({'label': str(year)})

    image_list = [create_evi_image(y) for y in years]
    gif_collection = ee.ImageCollection(image_list)

    gif_url = gif_collection.getVideoThumbURL({
        'dimensions': 400,
        'region': region.bounds(),
        'framesPerSecond': 1,
        'format': 'gif'
    })

    st.subheader("🎮 EVI Animation Over Selected Region")
    st.image(gif_url, caption="🌿 Animated EVI Change (2016–2025)", width=400)

    st.markdown("### 📊 Yearly EVI Snapshots")
    thumbs = []
    for year in years:
        evi = ee.ImageCollection("MODIS/061/MOD13Q1") \
            .filterDate(f"{year}-01-01", f"{year}-12-31") \
            .filterBounds(region).select("EVI").mean() \
            .multiply(0.0001).clip(region)

        vis = {
            "min": 0,
            "max": 0.8,
            "palette": ['#ffffcc', '#c2e699', '#78c679', '#31a354', '#006837']
        }

        url = evi.visualize(**vis).getThumbURL({
            "region": region.bounds().getInfo(),
            "dimensions": 150,
            "format": "png"
        })
        thumbs.append((year, url))

    cols = st.columns(len(thumbs))
    for i, (yr, url) in enumerate(thumbs):
        with cols[i]:
            st.image(url, caption=str(yr), use_container_width=True)

# ------------------ Page 3 ------------------
if page == "Environmental Statistics":
    st.header("📊 Environmental Statistics")

    param_bands = {
        "NDVI": ("MODIS/061/MOD13Q1", "NDVI", 250, 0.0001),
        "EVI": ("MODIS/061/MOD13Q1", "EVI", 250, 0.0001),
        "Precipitation": ("UCSB-CHG/CHIRPS/DAILY", "precipitation", 5000, 1),
        "LST (°C)": ("MODIS/061/MOD11A1", "LST_Day_1km", 1000, 0.02),
        "Tree Cover (%)": ("MODIS/061/MOD44B", "Percent_Tree_Cover", 250, 1)
    }

    start_year = 2015
    end_year = 2023
    data = {k: [] for k in param_bands}
    labels = []

    for y in range(start_year, end_year + 1):
        start = f"{y}-01-01"
        end = f"{y}-12-31"
        labels.append(str(y))
        for name, (ic, band, scale, factor) in param_bands.items():
            coll = ee.ImageCollection(ic).filterDate(start, end).filterBounds(region)
            img = coll.select(band).mean()
            if name == "LST (°C)":
                img = img.multiply(factor).subtract(273.15)
            elif factor != 1:
                img = img.multiply(factor)
            val = img.reduceRegion(ee.Reducer.mean(), region, scale).get(band).getInfo()
            data[name].append(val if val is not None else 0)

    df = pd.DataFrame(data, index=labels)

    # ---- Graph 1: EVI vs Precipitation (Normalized) ----
    st.markdown("### 📈 Normalized EVI vs Precipitation (2015–2023)")
    norm_df = df[['EVI', 'Precipitation']].copy()
    norm_df = (norm_df - norm_df.min()) / (norm_df.max() - norm_df.min())

    fig1, ax1 = plt.subplots(figsize=(6, 4))
    ax1.plot(labels, norm_df['EVI'], marker='o', label="EVI")
    ax1.plot(labels, norm_df['Precipitation'], marker='s', label="Precipitation")
    ax1.set_ylabel("Normalized Value")
    ax1.set_xlabel("Year")
    ax1.set_title("EVI vs Precipitation")
    ax1.legend()
    ax1.grid(True)
    st.pyplot(fig1)

    # ---- Graph 2: Correlation Matrix ----
    st.markdown("### 📊 Correlation Matrix")
    corr = df.corr()
    fig2, ax2 = plt.subplots(figsize=(6, 4))
    im = ax2.imshow(corr, cmap='coolwarm', vmin=-1, vmax=1)
    ax2.set_xticks(np.arange(len(corr.columns)))
    ax2.set_yticks(np.arange(len(corr.columns)))
    ax2.set_xticklabels(corr.columns, rotation=45, ha="right")
    ax2.set_yticklabels(corr.columns)
    for i in range(len(corr.columns)):
        for j in range(len(corr.columns)):
            ax2.text(j, i, f"{corr.iloc[i, j]:.2f}", ha="center", va="center", color="black")
    fig2.colorbar(im, ax=ax2)
    ax2.set_title("Correlation Matrix")
    st.pyplot(fig2)

    # ---- Graph 3: Monthly Trends (Normalized) ----
    st.markdown("### 📅 Monthly Trends of EVI and Precipitation")
    monthly_df = pd.DataFrame(columns=["Month", "EVI", "Precipitation"])

    for month in range(1, 13):
        start = f"2023-{month:02d}-01"
        end = f"2023-{month:02d}-28"
        evi = ee.ImageCollection("MODIS/061/MOD13Q1").filterDate(start, end).filterBounds(region).select("EVI").mean().multiply(0.0001)
        precip = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate(start, end).filterBounds(region).select("precipitation").sum()

        evi_val = evi.reduceRegion(ee.Reducer.mean(), region, 250).get("EVI").getInfo()
        precip_val = precip.reduceRegion(ee.Reducer.mean(), region, 5000).get("precipitation").getInfo()

        monthly_df.loc[month - 1] = [month, evi_val or 0, precip_val or 0]

    monthly_df["EVI_norm"] = (monthly_df["EVI"] - monthly_df["EVI"].min()) / (monthly_df["EVI"].max() - monthly_df["EVI"].min())
    monthly_df["Precip_norm"] = (monthly_df["Precipitation"] - monthly_df["Precipitation"].min()) / (monthly_df["Precipitation"].max() - monthly_df["Precipitation"].min())

    fig3, ax3 = plt.subplots(figsize=(6, 4))
    ax3.plot(monthly_df["Month"], monthly_df["EVI_norm"], marker='o', label="EVI (Normalized)")
    ax3.plot(monthly_df["Month"], monthly_df["Precip_norm"], marker='s', label="Precipitation (Normalized)")
    ax3.set_xticks(range(1, 13))
    ax3.set_xlabel("Month")
    ax3.set_ylabel("Normalized Value")
    ax3.set_title("Monthly EVI & Precipitation (2023)")
    ax3.grid(True)
    ax3.legend()
    st.pyplot(fig3)


'''

In [132]:
with open("app9.py", "w", encoding="utf-8") as f:
    f.write(code9)

In [133]:
!wget -q -O - ipv4.icanhazip.com

35.237.115.225


In [136]:
!streamlit run app9.py & npx localtunnel --port 8501

⠙⠹

⠸⠼⠴⠦⠧⠇⠏
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.115.225:8501

your url is: https://forty-chefs-mate.loca.lt
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
  Stopping...
^C


In [ ]:
code10 = '''

'''